# EFM First-Principles Simulation: Dynamic Parameter Evolution

## Objective: A Foundational Test of the Ehokolo Fluxon Model

This notebook represents a pivotal, first-principles test of the EFM's core theoretical claims. Unlike previous simulations that used fixed parameters for specific physical regimes (e.g., LSS, mass generation), this simulation embeds the **axiomatically-derived functional forms** of the EFM parameters directly into the core dynamics.

The central hypothesis being tested is **spontaneous self-organization**. By making the stability parameter (`m`) and the emergence parameter (`g`) direct functions of the local field density (`ρ`), we will test if the EFM field, starting from a simple initial state, naturally evolves and differentiates into the distinct, complex structures corresponding to the **S/T (cosmic)**, **T/S (quantum)**, and **S=T (resonant)** states.

**Key Theoretical Implementations:**
1.  **Dynamic Stability `m(ρ)`:** The stability term `m²` is calculated at every point in space and time via the derived relation `m² ∝ ρ`.
2.  **Dynamic Emergence `g(ρ)`:** The self-interaction term `g` is also calculated dynamically via `g ∝ ρ`. Crucially, its sign is determined by a density threshold, allowing the system to choose between attractive (`g < 0`, particle-like) and repulsive (`g > 0`, cosmic-like) self-interaction based on local conditions.
3.  **JIT Compilation:** The entire dynamic update step is encapsulated in a JIT-compiled function for maximum computational performance on the `512³` grid.

A successful run will show a non-uniform final state where regions of high density have generated particle-like parameters (high `m`, negative `g`) and regions of low density have generated cosmic-like parameters (low `m`, positive `g`). This would provide powerful evidence that the EFM is a truly fundamental, self-organizing theory.

In [ ]:
import os
import torch
import torch.nn.functional as F
import gc
from tqdm.notebook import tqdm
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt

try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
except ImportError:
    print("Not in Google Colab environment.")

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print(f"Using GPU: {torch.cuda.get_device_name(device)}, VRAM: {torch.cuda.get_device_properties(device).total_memory / 1e9:.2f} GB")
else:
    device = torch.device('cpu')
    print("No GPU available, running on CPU.")

data_path_dynamic = '/content/drive/My Drive/EFM_Simulations/data/FirstPrinciples_Dynamic_N512/'
os.makedirs(data_path_dynamic, exist_ok=True)
print(f"Dynamic Parameter Simulation Data will be saved to: {data_path_dynamic}")

In [ ]:
config = {
    'N': 512,
    'L_sim_unit': 40.0, # Larger box to allow for diverse structures
    'T_steps': 100000,
    'dt_cfl_factor': 0.001,
    'c_sim_unit': 1.0,

    # Base EFM Parameters (from other simulations, now used as references)
    'k_efm_gravity_coupling': 0.005,
    'eta_sim': 0.01,
    'alpha_sim': 0.1, # Using a moderate, stable alpha
    'delta_sim': 0.0002,

    # Reference parameters for dynamic calculation
    'rho_ref': 1.5,      # Reference density, corresponds to n'=1 HDS
    'm_ref': 1.0,        # Stability parameter at rho_ref (m²=1)
    'g_ref': 0.1,        # Emergence parameter magnitude at rho_ref
    'g_sign_threshold': 1.0, # Density threshold to flip sign of g (S/T vs S=T)

    # Initial conditions
    'initial_perturbation_amplitude': 15.0, # A strong central pulse to kickstart dynamics
    'initial_perturbation_width': 2.0,
    'background_noise_amplitude': 1.0e-4, # Low-level noise everywhere

    'history_every_n_steps': 500
}

config['dx_sim_unit'] = config['L_sim_unit'] / config['N']
config['dt_sim_unit'] = config['dt_cfl_factor'] * config['dx_sim_unit'] / config['c_sim_unit']
config['run_id'] = f"DynamicParams_N{config['N']}_T{config['T_steps']}"

print(f"--- EFM First-Principles Simulation Configuration ({config['run_id']}) ---")
for key, value in config.items():
    print(f"{key}: {value}")

In [ ]:
@torch.jit.script
def conv_laplacian_gpu(phi_field: torch.Tensor, dx: float) -> torch.Tensor:
    stencil = torch.tensor([[[0.,0.,0.],[0.,1.,0.],[0.,0.,0.]],[[0.,1.,0.],[1.,-6.,1.],[0.,1.,0.]],[[0.,0.,0.],[0.,1.,0.],[0.,0.,0.]]], 
                           dtype=phi_field.dtype, device=phi_field.device) / (dx**2)
    stencil = stencil.view(1, 1, 3, 3, 3)
    phi_padded = F.pad(phi_field.unsqueeze(0).unsqueeze(0), (1,1,1,1,1,1), mode='circular')
    return F.conv3d(phi_padded, stencil, padding=0).squeeze(0).squeeze(0)

@torch.jit.script
def nlkg_derivative_dynamic_gpu(phi: torch.Tensor, phi_dot: torch.Tensor, 
                                k_gravity: float, eta: float, c_sq: float, alpha: float, 
                                delta: float, dx: float, rho_ref: float, m_ref_sq: float, 
                                g_ref: float, g_sign_thresh: float) -> tuple[torch.Tensor, torch.Tensor]:
    
    phi_f32 = phi.to(torch.float32)
    phi_dot_f32 = phi_dot.to(torch.float32)

    # --- DYNAMIC PARAMETER CALCULATION ---
    rho_local = k_gravity * phi_f32**2 + 1e-12 # Add epsilon for stability
    
    # m(ρ)² ∝ ρ  => m_sq = m_ref_sq * (ρ / ρ_ref)
    m_sq_field = m_ref_sq * (rho_local / rho_ref)
    
    # g(ρ) ∝ ρ => g = g_ref * (ρ / ρ_ref)
    g_field_unsigned = g_ref * (rho_local / rho_ref)
    
    # Determine sign of g based on density threshold
    sign_field = torch.where(rho_local > g_sign_thresh, -1.0, 1.0)
    g_field_signed = sign_field * g_field_unsigned
    
    # --- STANDARD NLKG TERMS ---
    lap_phi = conv_laplacian_gpu(phi_f32, dx)
    
    # Use the DYNAMICALLY calculated m and g fields
    potential_force = m_sq_field * phi_f32 + g_field_signed * torch.pow(phi_f32, 3) + eta * torch.pow(phi_f32, 5)
    
    grad_phi_x = (torch.roll(phi_f32, shifts=-1, dims=0) - torch.roll(phi_f32, shifts=1, dims=0)) / (2 * dx)
    grad_phi_y = (torch.roll(phi_f32, shifts=-1, dims=1) - torch.roll(phi_f32, shifts=1, dims=1)) / (2 * dx)
    grad_phi_z = (torch.roll(phi_f32, shifts=-1, dims=2) - torch.roll(phi_f32, shifts=1, dims=2)) / (2 * dx)
    grad_phi_abs_sq = grad_phi_x**2 + grad_phi_y**2 + grad_phi_z**2

    alpha_term = alpha * phi_f32 * phi_dot_f32 * grad_phi_abs_sq
    delta_term = delta * torch.pow(phi_dot_f32, 2) * phi_f32

    phi_ddot = c_sq * lap_phi - potential_force + alpha_term - delta_term # Note: Corrected sign for delta dissipation
    
    return phi_dot, phi_ddot.to(phi.dtype)

@torch.jit.script
def update_phi_rk4_dynamic_gpu(phi_current, phi_dot_current, dt, config, dx):
    # Unpack config values inside the JIT function
    k_gravity, eta, c_sq, alpha = config[0], config[1], config[2], config[3]
    delta, rho_ref, m_ref_sq, g_ref, g_sign_thresh = config[4], config[5], config[6], config[7], config[8]

    k1_v, k1_a = nlkg_derivative_dynamic_gpu(phi_current, phi_dot_current, k_gravity, eta, c_sq, alpha, delta, dx, rho_ref, m_ref_sq, g_ref, g_sign_thresh)
    k2_v, k2_a = nlkg_derivative_dynamic_gpu(phi_current + 0.5*dt*k1_v, phi_dot_current + 0.5*dt*k1_a, k_gravity, eta, c_sq, alpha, delta, dx, rho_ref, m_ref_sq, g_ref, g_sign_thresh)
    k3_v, k3_a = nlkg_derivative_dynamic_gpu(phi_current + 0.5*dt*k2_v, phi_dot_current + 0.5*dt*k2_a, k_gravity, eta, c_sq, alpha, delta, dx, rho_ref, m_ref_sq, g_ref, g_sign_thresh)
    k4_v, k4_a = nlkg_derivative_dynamic_gpu(phi_current + dt*k3_v, phi_dot_current + dt*k3_a, k_gravity, eta, c_sq, alpha, delta, dx, rho_ref, m_ref_sq, g_ref, g_sign_thresh)

    phi_next = phi_current + (dt / 6.0) * (k1_v + 2*k2_v + 2*k3_v + k4_v)
    phi_dot_next = phi_dot_current + (dt / 6.0) * (k1_a + 2*k2_a + 2*k3_a + k4_a)
    return phi_next, phi_dot_next

print("JIT-Optimized Dynamic Parameter simulation functions defined.")

In [ ]:
if __name__ == '__main__':
    print("--- INITIATING FIRST-PRINCIPLES DYNAMIC PARAMETER SIMULATION ---")
    torch.manual_seed(42)
    
    # --- Initialize Fields Directly on GPU ---
    coords = torch.linspace(-config['L_sim_unit']/2, config['L_sim_unit']/2, config['N'], device=device)
    X, Y, Z = torch.meshgrid(coords, coords, coords, indexing='ij')
    r_sq = X**2 + Y**2 + Z**2
    
    # Initial state: Strong central pulse + low-level background noise
    central_pulse = config['initial_perturbation_amplitude'] * torch.exp(-r_sq / (config['initial_perturbation_width']**2))
    noise = torch.rand_like(X) * config['background_noise_amplitude']
    phi_current = (central_pulse + noise).to(torch.float16)
    phi_dot_current = torch.zeros_like(phi_current, dtype=torch.float16)
    del X, Y, Z, r_sq, coords, central_pulse, noise; gc.collect(); torch.cuda.empty_cache()

    # --- Pack config into a tensor for JIT --- 
    config_tensor = torch.tensor([
        config['k_efm_gravity_coupling'], config['eta_sim'], config['c_sim_unit']**2, config['alpha_sim'],
        config['delta_sim'], config['rho_ref'], config['m_ref']**2, config['g_ref'], config['g_sign_threshold']
    ], device=device, dtype=torch.float32)

    # --- Prepare History Arrays ---
    num_hist_points = config['T_steps'] // config['history_every_n_steps'] + 1
    energy_history = np.zeros(num_hist_points)
    max_phi_history = np.zeros(num_hist_points)
    history_idx = 0

    # --- Simulation Loop ---
    pbar = tqdm(range(config['T_steps']), desc=f"Dynamic Sim ({config['N']}³)")
    sim_start_time = time.time()

    for t_step in pbar:
        phi_current, phi_dot_current = update_phi_rk4_dynamic_gpu(
            phi_current, phi_dot_current, config['dt_sim_unit'], config_tensor, config['dx_sim_unit']
        )

        if (t_step + 1) % config['history_every_n_steps'] == 0:
            if torch.any(torch.isinf(phi_current)) or torch.any(torch.isnan(phi_current)):
                print(f"\nERROR: NaN/Inf detected at step {t_step + 1}! Halting."); break
            
            max_phi = torch.max(torch.abs(phi_current)).item()
            max_phi_history[history_idx] = max_phi
            history_idx += 1
            pbar.set_postfix({'Max|φ|': f'{max_phi:.3e}'})
            if max_phi > 1e7: print(f"\nWarning: Possible instability. Max|φ|={max_phi:.2e}"); break
    
    sim_duration = time.time() - sim_start_time
    print(f"Simulation finished in {sim_duration:.2f} seconds.")

    # --- Save Final State --- 
    final_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    final_data_filename = os.path.join(data_path_dynamic, f"FINAL_DATA_{config['run_id']}_{final_timestamp}.npz")
    np.savez_compressed(final_data_filename, phi_final_cpu=phi_current.cpu().numpy(), config=config)
    print(f"Final simulation state saved to {final_data_filename}")

    # --- Cleanup --- 
    del phi_current, phi_dot_current, config_tensor; gc.collect(); torch.cuda.empty_cache()
    print("\n--- SIMULATION COMPLETE. ANALYSIS WILL FOLLOW. ---")

## Analysis of Self-Organized Structures

This block loads the final state from the simulation and visualizes the key fields: the final scalar field `φ`, and the emergent fields for the stability parameter `m²(ρ)` and the emergence parameter `g(ρ)`. The plots of `m²` and `g` are the crucial result, showing whether the system self-organized into distinct regions with different physical laws as predicted.

In [ ]:
def analyze_dynamic_results(data_file_path, config):
    print(f"\n--- Analyzing Final State from {os.path.basename(data_file_path)} ---")
    data = np.load(data_file_path)
    phi_final = data['phi_final_cpu'].astype(np.float32)
    N = config['N']

    # Recalculate the emergent parameter fields from the final phi state
    rho_local = config['k_efm_gravity_coupling'] * phi_final**2 + 1e-12
    m_sq_field = (config['m_ref']**2) * (rho_local / config['rho_ref'])
    
    g_field_unsigned = config['g_ref'] * (rho_local / config['rho_ref'])
    sign_field = np.where(rho_local > config['g_sign_threshold'], -1.0, 1.0)
    g_field_signed = sign_field * g_field_unsigned

    # --- Visualization --- 
    # We will take a 2D slice through the center of the 3D cube
    center_slice = N // 2
    
    fig, axes = plt.subplots(1, 3, figsize=(24, 7))
    
    # Plot 1: Final Field φ
    im1 = axes[0].imshow(phi_final[center_slice, :, :], cmap='viridis', origin='lower')
    axes[0].set_title(f'Final Field φ (slice z={center_slice})', fontsize=16)
    fig.colorbar(im1, ax=axes[0], orientation='vertical', fraction=0.046, pad=0.04)
    
    # Plot 2: Emergent Stability Field m²(ρ)
    im2 = axes[1].imshow(m_sq_field[center_slice, :, :], cmap='magma', origin='lower')
    axes[1].set_title(f'Emergent Stability Field m²(ρ) (slice z={center_slice})', fontsize=16)
    fig.colorbar(im2, ax=axes[1], orientation='vertical', fraction=0.046, pad=0.04)
    
    # Plot 3: Emergent Self-Interaction Field g(ρ)
    # Use a diverging colormap to show the sign change
    g_max_abs = np.max(np.abs(g_field_signed))
    im3 = axes[2].imshow(g_field_signed[center_slice, :, :], cmap='coolwarm', vmin=-g_max_abs, vmax=g_max_abs, origin='lower')
    axes[2].set_title(f'Emergent Interaction Field g(ρ) (slice z={center_slice})', fontsize=16)
    fig.colorbar(im3, ax=axes[2], orientation='vertical', fraction=0.046, pad=0.04)
    
    fig.suptitle(f"First-Principles Self-Organization Results ({config['run_id']})", fontsize=20, y=1.02)
    plt.tight_layout()
    plt.savefig(os.path.join(data_path_dynamic, f"analysis_{config['run_id']}.png"))
    plt.show()

    # --- Quantitative Analysis ---
    g_positive_region = np.sum(g_field_signed > 0)
    g_negative_region = np.sum(g_field_signed < 0)
    total_cells = N**3
    print("\n--- Quantitative Analysis of Self-Organization ---")
    print(f"Fraction of volume with repulsive g>0 (S/T-like): {g_positive_region/total_cells:.2%}")
    print(f"Fraction of volume with attractive g<0 (S=T-like): {g_negative_region/total_cells:.2%}")
    print(f"Max emergent stability m²: {np.max(m_sq_field):.3e}")
    print(f"Min emergent stability m²: {np.min(m_sq_field):.3e}")
    print(f"Max |g|: {np.max(np.abs(g_field_signed)):.3e}")

if __name__ == '__main__':
    # Assuming the simulation has finished and `final_data_filename` is defined.
    # For a standalone run, you would manually set this path.
    # final_data_filename = '/content/drive/My Drive/EFM_Simulations/data/FirstPrinciples_Dynamic_N512/FINAL_DATA_...npz'
    if 'final_data_filename' in locals() and os.path.exists(final_data_filename):
        analyze_dynamic_results(final_data_filename, config)
    else:
        print("\nAnalysis skipped. Run the simulation cell first or provide a valid path to a data file.")